In [ ]:
import glob
import os
import argparse
from Bio import PDB
from Bio.PDB.Polypeptide import is_aa
import numpy as np
import pandas as pd
from pathlib import Path
import tqdm

def parse_args():
    parser = argparse.ArgumentParser(description="Filter out active site residues from a list of residues")
    parser.add_argument('-dir', '--dock_results', required=True, help='Path to file containing all the docking results')
    parser.add_argument('-meta', '--metadata', type=str, required=True, help='each seq metadata file with the active site residues')
    parser.add_argument('-atoms', '--ester_bond_atoms', nargs='+', type=int, default=[8,19], required=True, help='List of wanted atoms in the ester bonds of the substrate - defined by atom number in PDB file')
    parser.add_argument('-temps', '--templates', type=str, help='dir with template PDBs with known activity for conformation comparisons')
    return parser.parse_args()

def load_S_OG(pdb_file, residue_numbers):
    SER = None
    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('ATOM'):
                atom_info = line.split()
                if atom_info[2] == 'OG'and atom_info[3] == 'SER' and int(atom_info[5]) in residue_numbers:
                    SER = atom_info
    return SER

def parse_ligand_bond_atoms(pdb_file, atoms):
    with open(pdb_file, 'r') as f:
        ester_bond_atoms = []
        for line in f:
            if line.startswith('MODEL'):
                model = line.split()[1]
            if line.startswith('REMARK'):
                if line.split()[1] == 'VINA' and line.split()[2] == 'RESULT:':
                    result = line.split()
            if line.startswith('ATOM'):
                atom_info = line.split()
                if int(atom_info[1]) in atoms:
                    ester_bond_atoms.append((model,result,atom_info))
    ester_bond_dict = {}     # make a dictionary of ester bond atoms for each model
    for model, result, atom_info in ester_bond_atoms:
        if model not in ester_bond_dict:
            ester_bond_dict[model] = []
            ester_bond_dict[model].append(result)
        ester_bond_dict[model].append(atom_info)
    return ester_bond_dict

def atom_distance(Serine, atom2):
    x1, y1, z1 = float(Serine[6]), float(Serine[7]), float(Serine[8])
    x2, y2, z2 = float(atom2[5]), float(atom2[6]), float(atom2[7])
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return distance

def measure_all_distances(serine, ester_bonds):
    model_distances = []
    for model in ester_bonds:
        distances = []
        for atom in ester_bonds[model][1:]:
            distances.append(atom_distance(serine, atom))
        model_distances.append((model, (ester_bonds[model][0])[3:], distances, ester_bonds[model][1:]))
    # make a df of the results with columns: model, distances, atoms
    results = pd.DataFrame(model_distances, columns=['model', 'result', 'distances', 'atoms'])
    return results

def get_residues_by_position(structure, positions):
    residues = []
    for model in structure:
        for chain in model:
            seq_residues = [res for res in chain if is_aa(res)]
            selected_residues = [seq_residues[pos - 1] for pos in positions]  # Position is 1-based
            residues.extend(selected_residues)
            if len(residues) == len(positions):
                break
    return residues

def get_atoms_from_residues(residues):
    """
    Retrieves all atoms from the given residues.
    """
    # If single residues passed, just put in list and continue. Lazy, I know.
    if not isinstance(residues, list):
        residues = [residues]
    
    atoms = []
    for residue in residues:
        atoms.extend(residue.get_atoms())
    return atoms

def extract_backbone_atoms(residues):
    """
    Retrieves only the backbone atoms from the given residues. Useful if active site residues being compared are different.
    Backbones are always the same regardless of residue.
    """
    # If single residues passed, just put in list and continue. Lazy, I know.
    if not isinstance(residues, list):
        residues = [residues]
    backbone_atoms = []
    for residue in residues:
        for atom_name in ["N", "CA", "C", "O"]:
            backbone_atoms.append(residue[atom_name])
    return backbone_atoms
    
def save_superimposed_residues(residues1, residues2, output_pdb):
    # Create a new structure with the superimposed residues
    new_structure = PDB.Structure.Structure("Superimposed")
    model = PDB.Model.Model(0)
    chain1 = PDB.Chain.Chain("A")
    chain2 = PDB.Chain.Chain("B")
    for res in residues1:
        chain1.add(res.copy())
    for res in residues2:
        chain2.add(res.copy())
    model.add(chain1)
    model.add(chain2)
    new_structure.add(model)
    # Save the new structure to a PDB file
    io = PDB.PDBIO()
    io.set_structure(new_structure)
    io.save(output_pdb)

def Ser_ester_bond_distance(enzyme, residues, substrate, ester_bond_atoms):
    Ser = load_S_OG(enzyme, residues)
    ester_bond_dict = parse_ligand_bond_atoms(substrate, ester_bond_atoms)
    distances = measure_all_distances(Ser, ester_bond_dict)
    return distances

def superimpose_residues(pdb1, pdb2, residues1, residues2):
    parser = PDB.PDBParser(QUIET=True)
    structure1 = parser.get_structure('structure1', pdb1)
    structure2 = parser.get_structure('structure2', pdb2)
    residues1 = get_residues_by_position(structure1, residues1)
    residues2 = get_residues_by_position(structure2, residues2)
    # Reorder the residues so they are in the same order
    residues1 = sorted(residues1, key=lambda x: x.resname)
    residues2 = sorted(residues2, key=lambda x: x.resname)
    # Reorder the residues so Serine is first, Histadine is second and the rest follow
    serine1 = [res for res in residues1 if res.resname == 'SER']
    histidine1 = [res for res in residues1 if res.resname == 'HIS']
    serine2 = [res for res in residues2 if res.resname == 'SER']
    histidine2 = [res for res in residues2 if res.resname == 'HIS']
    other1 = [res for res in residues1 if res not in serine1 and res not in histidine1]
    other2 = [res for res in residues2 if res not in serine2 and res not in histidine2]
    residues1 = serine1 + histidine1 + other1
    residues2 = serine2 + histidine2 + other2
    # Now we check if the list of residues are equivalent and then create the boolean list to select atoms to superimpose (backbone or chain)
    if len(residues1) > 3 and len(residues2) > 3:
        # raise error
        raise ValueError('Different number of residues in template and query active sites!')
    else:
        backbone_bool = []
        for i in range(len(residues1)):
            res1 = residues1[i]
            res2 = residues2[i]
            if res1.resname == res2.resname:
                backbone_bool.append(False)
            else:
                backbone_bool.append(True)
    # We need to just compare backbones of other residues here so we can accomodate for Glu and Asp in the third residues
    # We cannot superimpose the side chains of Glu and Asp because they have different atoms
    atoms1 = []
    atoms2 = []
    index = 0
    for b in backbone_bool:
        if b == True:
            atoms1.extend(extract_backbone_atoms(residues1[index]))
            atoms2.extend(extract_backbone_atoms(residues2[index]))
        else:
            atoms1.extend(get_atoms_from_residues(residues1[index]))
            atoms2.extend(get_atoms_from_residues(residues2[index]))
        index+=1
    super_imposer = PDB.Superimposer()
    super_imposer.set_atoms(atoms1, atoms2)
    super_imposer.apply(structure2)
    # Retrieve the transformed residues
    # residues2 = get_residues_by_position(structure2, residues2)
    return super_imposer.rms

def main():
    args = parse_args()
    #Using glob, get all subdirectories in the results directory
    subdirs = glob.glob(f'{args.dock_results}/*')
    subdirs = [Path(x) for x in subdirs]
    # check that the files are directories
    subdirs = [x for x in subdirs if x.is_dir()]
    metadata = pd.read_csv(Path(args.metadata))
    # init tqdm
    subdirs = tqdm.tqdm(subdirs)
    for dir in subdirs:
        #############################
        # --- Ser Bond Distance --- #
        #############################
        ligand_pdb = glob.glob(f'{dir}/*_docked_ligand.pdb')
        ligand_pdb = Path(ligand_pdb[0])
        enzyme = Path(dir / f"{dir.name}.pdb")
        
        enzyme_metadata = metadata[metadata['Entry'] == dir.name]
        AS = enzyme_metadata['Residue'].values[0].split('|')
        AS = [int(x) for x in AS]
        
        dist_df = Ser_ester_bond_distance(enzyme, AS, ligand_pdb, args.ester_bond_atoms)
        
        # Save dist_df in dir as a tsv
        dist_df.to_csv(f'{dir}/SER-BOND_distances.tsv', sep='\t', index=False)
        
        ####################################
        # --- Conformation comparisons --- #
        ####################################
        # Get the template PDBs
        templates = glob.glob(f'{args.templates}/*.pdb')
        templates = [Path(x) for x in templates]
        template_residues = pd.read_csv((Path(args.templates) / 'template_residues.tsv'), sep='\t')
        RMSDs = []
        for template in templates:
            template_AS = template_residues[template_residues['Entry'] == template.name[:-4]]['Residues']
            template_AS = template_AS.values[0].split('|')
            template_AS = [int(x) for x in template_AS]
            RMSD = superimpose_residues(template, enzyme, template_AS, AS)
            RMSDs.append(RMSD)
        # Make a df with the RMSDs and their corresponding pdb name
        templates = [x.name[:-4] for x in templates]
        RMSD_df = pd.DataFrame({'template': templates, 'RMSD': RMSDs})
        
        # Save RMSD_df in dir as a tsv
        RMSD_df.to_csv(f'{dir}/RMSD_values.tsv', sep='\t', index=False)

if __name__ == '__main__':
    main()

In [ ]:


from Bio.PDB import PDBParser
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole  # Enables RDKit drawings in Jupyter notebooks
from pathlib import Path
import pandas as pd
from rdkit import Chem
from tqdm import tqdm
import argparse
# !pip install PubChemPy rdkit
# For each of these map the smiles
import pubchempy as pcp
from rdkit import Chem

# Permute each one so that we can see if this has an effect on whether we can pull out the enzymes! 
import itertools

from rdkit import Chem
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole  # Enables RDKit drawings in Jupyter notebooks


# Now expand out to move substrates and products around
def permute_substrates_products(substrates, products, reversible=False):
    """ make all permutatations of substrates and products """
    substrate_combinations = list(itertools.permutations(substrates))
    product_combinations = list(itertools.permutations(products))
    product_combinations.append(products)
    reactions = []
    for substrate in substrate_combinations:
        for product in product_combinations:
            combo = f'{".".join(substrate)}>>{".".join(product)}'
            if combo not in reactions:
                reactions.append(combo)
    
    return reactions

def cleave_ester_bonds(substrate_smiles, vis=False):
    # We're going to "clean" the string for this we're assuming that we're just going to be breaking the biggest substrate
    substrates = substrate_smiles.split('.')
    longest = None
    for s in substrates:
        if longest is None:
            longest = s
        elif len(s) > len(longest):
            longest = s
    substrate_smiles = longest
    # Create reactions based on cleaving ester bonds
    reactions = []
    # Convert the SMILES string to an RDKit molecule object
    mol = Chem.MolFromSmiles(substrate_smiles)
    
    # Define a SMARTS pattern for the ester bond (C=O bonded to O)
    ester_pattern = Chem.MolFromSmarts('C(=O)OC') # We get the second one so we know where to "cut" it
    
    # Find the substructure matches (ester bonds) in the molecule
    matches = mol.GetSubstructMatches(ester_pattern)
    
    # Flatten the list of matched atom indices (since they are tuples)
    highlight_atoms = [atom_idx for m in matches for atom_idx in m]
    
    highlight_cut = [m[2] for m in matches] #i.e. the third one
    
    for m in matches:
        # The third atom in the match tuple is the single-bonded oxygen
        oxygen_idx = m[2]
        # The second atom is the carbon that it's single-bonded to
        carbon_idx = m[3]
        
        mol_editable = Chem.EditableMol(mol)
        # Remove the bond between the oxygen and the carbon
        mol_editable.RemoveBond(carbon_idx, oxygen_idx)
        
        # Add a hydrogen atom to the new oxygen (to form OH)
        new_hydrogen_idx = mol_editable.AddAtom(Chem.Atom(1))  # 1 corresponds to hydrogen

        # Add an oxygen atom (for the OH group)
        new_oxygen_idx = mol_editable.AddAtom(Chem.Atom(8))  # 8 corresponds to oxygen
        mol_editable.AddBond(carbon_idx, new_oxygen_idx, Chem.BondType.SINGLE)  # Create a single bond to the oxygen

        # Also we want to add a hydrogen at the clevaed one as well!
        new_hydrogen_on_cleaved = mol_editable.AddAtom(Chem.Atom(1))  # 1 corresponds to hydrogen

        # Convert back to a normal molecule object
        cleaved_mol_with_oh = mol_editable.GetMol()
        # Optional: sanitize the molecule to clean up valences
        Chem.SanitizeMol(cleaved_mol_with_oh)
        
        cleaved_smiles = Chem.MolToSmiles(cleaved_mol_with_oh)
        
        cleaved_smiles = cleaved_smiles.replace('.[HH]', '')
        # In the future maybe only keep the longest one... for now we'll keep both
        reaction = f'{substrate_smiles}.O>>{cleaved_smiles}' # Add in 0 which is H20
        reactions.append(reaction)
        
        if vis == True:
            mol_image = Draw.MolToImage(mol, highlightAtoms=highlight_cut, highlightColor=(0.0, 1.0, 0.0))
            # Display the image
            mol_image.show()
            # Do the second one
            cleaved_mol_with_oh = Chem.MolFromSmiles(cleaved_smiles)
            mol_image = Draw.MolToImage(cleaved_mol_with_oh)
            mol_image.show()
        
    return reactions

def get_coordinates_without_chain_id(pdb_file, seq_position):
    # Create a PDB parser
    parser = PDBParser(QUIET=True)

    # Parse the PDB file
    structure = parser.get_structure("protein", pdb_file)

    # Iterate over all chains and residues to find the matching sequence position
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.id[1] == seq_position:
                    # Extract the coordinates of the alpha carbon (CA) atom
                    ca_atom = residue['CA']
                    return residue, ca_atom.get_coord()

    return None, None
    
def load_residues_from_pdb(pdb_file, resnames, res_positions=None):
    # Create a PDB parser
    parser = PDBParser(QUIET=True)

    # Parse the PDB file
    structure = parser.get_structure("protein", pdb_file)
    serines = []
    # Iterate over all chains and residues to find the matching sequence position
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.resname in resnames:
                    if res_positions is not None:
                        if residue.id[1] in res_positions:
                            ca_atom = residue['CA']  # ca_atom.get_coord()
                            serines.append([residue, ca_atom, ca_atom.get_coord()])

                    else:
                        # Extract the coordinates of the alpha carbon (CA) atom
                        ca_atom = residue['CA'] #ca_atom.get_coord()
                        serines.append([residue, ca_atom, ca_atom.get_coord()])
    return serines


def atom_distance(atom1, atom2):
    x1, y1, z1 = float(atom1[0]), float(atom1[1]), float(atom1[2])
    x2, y2, z2 = float(atom2[0]), float(atom2[1]), float(atom2[2])
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return distance


def measure_all_distances(atoms, ester_bonds):
    rows = []
    for atom in atoms:
        for bond in ester_bonds:
            a = atom[1]
            distance = atom_distance(a.coord, bond.coord)
            rows.append([a.parent.resname, atom[0], distance])
    # make a df of the results with columns: model, distances, atoms
    df = pd.DataFrame(rows, columns=['name', 'residue', 'distance'])
    return df


def calculate_distance(atom1, atom2):
    """Calculates the Euclidean distance between two atoms."""
    return np.linalg.norm(atom1 - atom2)


def find_esters(atoms, bond_cutoff=1.6):
    # Loop through atoms and identify carbon and oxygen atoms
    bonds = []
    for atom in atoms:
        distances = []
        if atom[2] == "C":
            # Find a carbon that is bonded to two oxygen atoms
            oxygens = []
            for other_atom in atoms:
                if other_atom[2] == "O":
                    atom_1_coords = np.array([float(x) for x in atom[5:8]])
                    atom_2_coords = np.array([float(x) for x in other_atom[5:8]])
                    distance = calculate_distance(atom_1_coords, atom_2_coords)
                    distances.append(distance)
                    if distance <= bond_cutoff:
                        oxygens.append([atom, other_atom])
            # # Check if there are exactly two oxygens bonded to this carbon
            if len(oxygens) == 2:
                carbonyl_oxygen, ester_oxygen = oxygens[0][0], oxygens[0][1]
                bonds += [ester_oxygen]
    return bonds


def find_ester_bonds_vina(pdb_file, bond_cutoff=1.6):
    oxygens = []
    on_model = False

    with open(pdb_file, 'r') as fin:
        atoms = []
        for line in fin:
            # First get any models that are in here if we're in a model then we just go for it
            if not on_model:
                if line.startswith('ATOM'):
                    on_model = True
                    # ATOM      1  C   UNL     1     -11.738  11.331   5.737  1.00  0.00           C
                    atoms.append([x for x in line.split(' ') if len(x) > 0])
            else:
                if line.startswith('ATOM'):
                    atoms.append([x for x in line.split(' ') if len(x) > 0])
                else:
                    on_model = False
                    # Now we want to go through the current atom list to see if there is a C-O-O bond
                    oxygens.append(find_esters(atoms, bond_cutoff))
                    atoms = []
    return oxygens

def find_ester_bonds(pdb_file, bond_cutoff=1.6):
    # Initialize the PDB parser
    parser = PDBParser(QUIET=True)
    bonds = []
    # Parse the structure from a PDB file
    structure = parser.get_structure("molecule", pdb_file)

    # Loop through all models, chains, and residues to find ester bonds
    for model in structure:
        atoms = []

        for chain in model:
            for residue in chain:
                atoms += [atom for atom in list(residue.get_atoms()) if atom.element in ['C', 'O']]
        # Loop through atoms and identify carbon and oxygen atoms
        for atom in atoms:
            if atom.parent.resname == 'LIG':
                distances = []
                if atom.element == "C":
                    # Find a carbon that is bonded to two oxygen atoms
                    oxygens = []
                    for other_atom in atoms:
                        if other_atom.element == "O":
                            distance = calculate_distance(atom.coord, other_atom.coord)
                            distances.append(distance)
                            if distance <= bond_cutoff:
                                oxygens.append([atom, other_atom])
                    # # Check if there are exactly two oxygens bonded to this carbon
                    if len(oxygens) == 2:
                        carbonyl_oxygen, ester_oxygen = oxygens[0][0], oxygens[0][1]
                        bonds += [atom]
    return bonds


def calculate_ester_nucleophile_distance(enzyme_pdb, ester_pdb, resnames, res_postions=None, chai=False):
    atoms = load_residues_from_pdb(enzyme_pdb, resnames, res_postions)
    # Initialize the PDB parser
    parser = PDBParser(QUIET=True)
    ester_bond_atoms = find_ester_bonds(ester_pdb)
    distances = measure_all_distances(atoms, ester_bond_atoms)
    return distances